In [1]:
# To be run after Adult-Data-Prep
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import balanced_accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
#from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_Adult
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Adult dataset
Adult_df = pd.read_csv('./input/adult-cleaned.csv')
# The AIF demo drops the following columns - we'll try the same
Adult_df.drop(["Fnlwgt"],axis=1,inplace=True)
#Adult_df.drop(["Fnlwgt", "NativeCountry", "Relationship", "MaritalStatus"],axis=1,inplace=True)

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'Gender'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'Income'
fav_label           = 1 # Income over £50K
unfav_label         = 0 # Income under £50K
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create the traoning and test splits
X = Adult_df.drop(lab,axis=1)
y = Adult_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
Adult_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Adult_train_bld, Adult_test_bld = Adult_bld.split([0.8], shuffle=True)

In [7]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
Adult_train_bld.features = min_max_scaler.fit_transform(Adult_train_bld.features)
Adult_test_bld.features  = min_max_scaler.fit_transform(Adult_test_bld.features)

In [8]:
# Convert the train and test datasets to dataframes
Adult_train_df, d = Adult_train_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df,  d = Adult_test_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [9]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Adult_train_df.drop(lab,axis=1)
y_train = Adult_train_df[lab]
X_test  = Adult_test_df.drop(lab,axis=1)
y_test  = Adult_test_df[lab]
BiasedLogModel = LogisticRegression(random_state=101)
BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
BiasedLogModel.fit(X_train, y_train) 
BiasedRfcModel.fit(X_train, y_train)
BiasedLogPredictions = BiasedLogModel.predict(X_test)
BiasedRfcPredictions = BiasedRfcModel.predict(X_test)
print(f"Biased Logistic regression validation accuracy: {BiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {BiasedRfcModel.score(X_test, y_test)}")
print('')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, BiasedLogPredictions)}")
print(f"Biased Random forest balanced accuracy        : {balanced_accuracy_score(y_test, BiasedRfcPredictions)}")

Biased Logistic regression validation accuracy: 0.8217831917289384
Biased Random Forest       validation accuracy: 0.8353976865595251

Biased Logistic regression balanced accuracy  : 0.6818849323466032
Biased Random forest balanced accuracy        : 0.6725080487787696


In [10]:
print('Training Before - Income value counts:')
print(Adult_train_df.Income.value_counts())
print('Training Before - Gender value counts:')
print(Adult_train_df.Gender.value_counts())

Training Before - Income value counts:
0.0    29713
1.0     9360
Name: Income, dtype: int64
Training Before - Gender value counts:
1.0    26052
0.0    13021
Name: Gender, dtype: int64


In [11]:
print('Test Before - Income value counts:')
print(Adult_test_df.Income.value_counts())
print('Test Before - Gender value counts:')
print(Adult_test_df.Gender.value_counts())

Test Before - Income value counts:
0.0    7442
1.0    2327
Name: Income, dtype: int64
Test Before - Gender value counts:
1.0    6598
0.0    3171
Name: Gender, dtype: int64


In [12]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(Adult_train_bld, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_test_bld = BinaryLabelDatasetMetric(Adult_test_bld, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

In [13]:
display(Markdown("#### Orig training dataset"))
print('Number of instances           :', metric_train_bld.num_instances())
print('Base Rate                     :', metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
display(Markdown("#### Orig test dataset"))
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

#### Orig training dataset

Number of instances           : 39073.0
Base Rate                     : 0.23955160852762777
Consistency                   : [0.83169452]
Disparate Impact              : 0.36168796895587446
Mean Difference               : -0.1942230719363881
Statistical Parity Difference : -0.1942230719363881


#### Orig test dataset

Number of instances           : 9769.0
Base Rate                     : 0.23820247722387142
Consistency                   : [0.8279046]
Disparate Impact              : 0.3511430576667853
Mean Difference               : -0.19579784361707042
Statistical Parity Difference : -0.19579784361707042


In [14]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [15]:
biased_model.fit(Adult_train_bld)

W0831 18:14:25.010267 11188 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0831 18:14:25.015812 11188 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0831 18:14:25.029087 11188 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0831 18:14:28.749302 11188 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.685152
epoch 0; iter: 200; batch classifier loss: 0.479433
epoch 1; iter: 0; batch classifier loss: 0.363894
epoch 1; iter: 200; batch classifier loss: 0.386594
epoch 2; iter: 0; batch classifier loss: 0.313558
epoch 2; iter: 200; batch classifier loss: 0.285010
epoch 3; iter: 0; batch classifier loss: 0.360125
epoch 3; iter: 200; batch classifier loss: 0.268008
epoch 4; iter: 0; batch classifier loss: 0.239669
epoch 4; iter: 200; batch classifier loss: 0.348645
epoch 5; iter: 0; batch classifier loss: 0.313359
epoch 5; iter: 200; batch classifier loss: 0.398899
epoch 6; iter: 0; batch classifier loss: 0.282975
epoch 6; iter: 200; batch classifier loss: 0.371400
epoch 7; iter: 0; batch classifier loss: 0.287612
epoch 7; iter: 200; batch classifier loss: 0.292009
epoch 8; iter: 0; batch classifier loss: 0.318573
epoch 8; iter: 200; batch classifier loss: 0.342842
epoch 9; iter: 0; batch classifier loss: 0.311289
epoch 9; iter: 200; batch classi

In [16]:
# Apply the plain model to test data
Adult_biased_train = biased_model.predict(Adult_train_bld)
Adult_biased_test  = biased_model.predict(Adult_test_bld)
# And convert them to dataframes
Adult_train_df, d = Adult_biased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df, d  = Adult_biased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [17]:
# Test whether it is possible to predict the Sensitive Variable from the biased dataset
X_se_train = Adult_train_df.drop(protected_attr,axis=1)
y_se_train = Adult_train_df[protected_attr]
X_se_test  = Adult_test_df.drop(protected_attr,axis=1)
y_se_test  = Adult_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7548367284266557
Biased Random Forest       validation accuracy: 0.7975227761285699

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.6893517898498336
Biased Random Forest       balanced accuracy  : 0.7554315122201437


In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_biased_train = BinaryLabelDatasetMetric(Adult_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_biased_test = BinaryLabelDatasetMetric(Adult_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_biased_train.num_instances())
print('Base Rate                     :', metric_biased_train.base_rate())
print('Consistency                   :', metric_biased_train.consistency())
print('Disparate Impact              :', metric_biased_train.disparate_impact())
print('Mean Difference               :', metric_biased_train.mean_difference())
print('Statistical Parity Difference :', metric_biased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_biased_test.num_instances())
print('Base Rate                     :', metric_biased_test.base_rate())
print('Consistency                   :', metric_biased_test.consistency())
print('Disparate Impact              :', metric_biased_test.disparate_impact())
print('Mean Difference               :', metric_biased_test.mean_difference())
print('Statistical Parity Difference :', metric_biased_test.statistical_parity_difference()) 

#### Plain model - without debiasing - dataset metrics

#### Biased training dataset

Number of instances           : 39073.0
Base Rate                     : 0.20356768100734524
Consistency                   : [0.95967036]
Disparate Impact              : 0.24350060294972123
Mean Difference               : -0.20590884479055455
Statistical Parity Difference : -0.20590884479055455


#### Biased test dataset

Number of instances           : 9769.0
Base Rate                     : 0.19920155594226635
Consistency                   : [0.94484594]
Disparate Impact              : 0.2802618114417006
Mean Difference               : -0.1870795685628196
Statistical Parity Difference : -0.1870795685628196


In [19]:
# Create the Classification metric for train and test
classified_metric_biased_test = ClassificationMetric(Adult_train_bld, Adult_biased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_biased_train = ClassificationMetric(Adult_test_bld, Adult_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [20]:
display(Markdown("#### Plain model - without debiasing - classification metrics"))

display(Markdown("#### Biased training dataset - CLassification Metrics"))
TPR_train = classified_metric_biased_train.true_positive_rate()
TNR_train = classified_metric_biased_train.true_negative_rate()
bal_acc_biased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_biased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_train)
print('Disparate impact                =', classified_metric_biased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_train.theil_index())

display(Markdown("#### Biased test dataset - CLassification Metrics"))
TPR_test = classified_metric_biased_test.true_positive_rate()
TNR_test = classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_biased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_test)
print('Disparate impact                =', classified_metric_biased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - classification metrics

#### Biased training dataset - CLassification Metrics

Classification accuracy         = 0.8529020370559934
Balanced classification accuracy= 0.7692098078167481
Disparate impact                = 0.2802618114417006
Equal opportunity difference    = -0.13826056540144943
Average odds difference         = -0.10763116070670366
Theil_index                     = 0.11774981149746311


#### Biased test dataset - CLassification Metrics

Classification accuracy         = 0.8531978604151205
Balanced classification accuracy= 0.7728095881511474
Disparate impact                = 0.24350060294972123
Equal opportunity difference    = -0.1658572189301346
Average odds difference         = -0.13006204357645484
Theil_index                     = 0.11632786221132833


In [21]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [22]:
unbiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [23]:
unbiased_model.fit(Adult_train_bld)

epoch 0; iter: 0; batch classifier loss: 0.680460; batch adversarial loss: 0.894226
epoch 0; iter: 200; batch classifier loss: 0.946332; batch adversarial loss: 1.011363
epoch 1; iter: 0; batch classifier loss: 0.852266; batch adversarial loss: 0.880331
epoch 1; iter: 200; batch classifier loss: 0.954274; batch adversarial loss: 0.720528
epoch 2; iter: 0; batch classifier loss: 0.701423; batch adversarial loss: 0.657317
epoch 2; iter: 200; batch classifier loss: 0.289072; batch adversarial loss: 0.652108
epoch 3; iter: 0; batch classifier loss: 0.441544; batch adversarial loss: 0.634345
epoch 3; iter: 200; batch classifier loss: 0.272201; batch adversarial loss: 0.676511
epoch 4; iter: 0; batch classifier loss: 0.328823; batch adversarial loss: 0.612028
epoch 4; iter: 200; batch classifier loss: 0.397757; batch adversarial loss: 0.657345
epoch 5; iter: 0; batch classifier loss: 0.262638; batch adversarial loss: 0.606627
epoch 5; iter: 200; batch classifier loss: 0.338500; batch adversa

epoch 48; iter: 0; batch classifier loss: 0.402862; batch adversarial loss: 0.684182
epoch 48; iter: 200; batch classifier loss: 0.355890; batch adversarial loss: 0.635136
epoch 49; iter: 0; batch classifier loss: 0.239760; batch adversarial loss: 0.691541
epoch 49; iter: 200; batch classifier loss: 0.351629; batch adversarial loss: 0.650936


In [24]:
Adult_unbiased_train = unbiased_model.predict(Adult_train_bld)
Adult_unbiased_test  = unbiased_model.predict(Adult_test_bld)
# And convert these to dataframes..
Adult_train_df, d = Adult_unbiased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Adult_test_df, d  = Adult_unbiased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [25]:
# Metrics for the dataset from plain model (after unbiasing)
display(Markdown("#### Plain model - after unbiasing - dataset metrics"))
metric_unbiased_train = BinaryLabelDatasetMetric(Adult_unbiased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_unbiased_test = BinaryLabelDatasetMetric(Adult_unbiased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_unbiased_train.num_instances())
print('Base Rate                     :', metric_unbiased_train.base_rate())
print('Consistency                   :', metric_unbiased_train.consistency())
print('Disparate Impact              :', metric_unbiased_train.disparate_impact())
print('Mean Difference               :', metric_unbiased_train.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_unbiased_test.num_instances())
print('Base Rate                     :', metric_unbiased_test.base_rate())
print('Consistency                   :', metric_unbiased_test.consistency())
print('Disparate Impact              :', metric_unbiased_test.disparate_impact())
print('Mean Difference               :', metric_unbiased_test.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_test.statistical_parity_difference()) 

#### Plain model - after unbiasing - dataset metrics

#### Biased training dataset

Number of instances           : 39073.0
Base Rate                     : 0.15806311263532363
Consistency                   : [0.95915338]
Disparate Impact              : 0.7974075323140213
Mean Difference               : -0.03434086969527414
Statistical Parity Difference : -0.03434086969527414


#### Biased test dataset

Number of instances           : 9769.0
Base Rate                     : 0.15579895588084758
Consistency                   : [0.94138602]
Disparate Impact              : 0.819335715267598
Mean Difference               : -0.029900787955105035
Statistical Parity Difference : -0.029900787955105035


In [26]:
# Create the Classification metric for train and test
classified_metric_unbiased_train = ClassificationMetric(Adult_train_bld, Adult_unbiased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_unbiased_test = ClassificationMetric(Adult_test_bld, Adult_unbiased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [27]:
display(Markdown("#### Plain model - after debiasing - classification metrics"))
display(Markdown("#### Unbiased training dataset - CLassification Metrics"))
TPR_train = classified_metric_unbiased_train.true_positive_rate()
TNR_train = classified_metric_unbiased_train.true_negative_rate()
bal_acc_unbiased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_unbiased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_train)
print('Disparate impact                =', classified_metric_unbiased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_train.theil_index())

display(Markdown("#### Unbiased test dataset - CLassification Metrics"))
TPR_test = classified_metric_unbiased_test.true_positive_rate()
TNR_test = classified_metric_unbiased_test.true_negative_rate()
bal_acc_unbiased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_unbiased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_test)
print('Disparate impact                =', classified_metric_unbiased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_test.theil_index())

#### Plain model - after debiasing - classification metrics

#### Unbiased training dataset - CLassification Metrics

Classification accuracy         = 0.8415785836767077
Balanced classification accuracy= 0.7243341086278101
Disparate impact                = 0.7974075323140213
Equal opportunity difference    = 0.2583792564231656
Average odds difference         = 0.13943432605596404
Theil_index                     = 0.14305764755686662


#### Unbiased test dataset - CLassification Metrics

Classification accuracy         = 0.8385709898659024
Balanced classification accuracy= 0.7181572156867139
Disparate impact                = 0.819335715267598
Equal opportunity difference    = 0.25034679868934007
Average odds difference         = 0.138550711455793
Theil_index                     = 0.14569295717927783


In [28]:
# Determine the after-transformation model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Adult_train_df.drop(lab,axis=1)
y_train = Adult_train_df[lab]
X_test  = Adult_test_df.drop(lab,axis=1)
y_test  = Adult_test_df[lab]

UnbiasedLogModel = LogisticRegression(random_state=101)
UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_train, y_train) 
UnbiasedRfcModel.fit(X_train, y_train) 
UnbiasedLog_pred = UnbiasedLogModel.predict(X_test)
UnbiasedRfc_pred = UnbiasedRfcModel.predict(X_test)

# Now get Logistic Regression and Random Forest Clasisfication performancce of unbiased data
print('Model Accuracy for Log Reg and RFC after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {UnbiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {UnbiasedRfcModel.score(X_test, y_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedRfc_pred)}")

Model Accuracy for Log Reg and RFC after bias transformation:
Biased Logistic regression validation accuracy: 0.9034701607124578
Biased Random Forest       validation accuracy: 0.9175964786569761

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.7480738426444882
Biased Random Forest       balanced accuracy  : 0.7403673011664975


In [29]:
# Finally test whether it is possible to predict the Sensitive Variable from the transformed dataset
X_se_train = Adult_train_df.drop(protected_attr,axis=1)
y_se_train = Adult_train_df[protected_attr]
X_se_test  = Adult_test_df.drop(protected_attr,axis=1)
y_se_test  = Adult_test_df[protected_attr]

Se_unbiasedLogModel = LogisticRegression(random_state=101)
Se_unbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_unbiasedLogModel.fit(X_se_train, y_se_train) 
Se_unbiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_unbiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_unbiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_unbiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_unbiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.7636400859862832
Biased Random Forest       validation accuracy: 0.7946565666905517

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.6898084327227013
Biased Random Forest       balanced accuracy  : 0.7525725712779185


In [30]:
print('Predicted Training After - Income value counts:')
print(Adult_train_df.Income.value_counts())
print('Predicted Training After - Gender value counts:')
print(Adult_train_df.Gender.value_counts())

Predicted Training After - Income value counts:
0.0    32897
1.0     6176
Name: Income, dtype: int64
Predicted Training After - Gender value counts:
1.0    26052
0.0    13021
Name: Gender, dtype: int64


In [31]:
print('Predicted Test After - Income value counts:')
print(Adult_test_df.Income.value_counts())
print('Predicted Test After - Gender value counts:')
print(Adult_test_df.Gender.value_counts())

Predicted Test After - Income value counts:
0.0    8247
1.0    1522
Name: Income, dtype: int64
Predicted Test After - Gender value counts:
1.0    6598
0.0    3171
Name: Gender, dtype: int64
